In [1]:
import pandas as pd
import os
from langchain_core.documents import Document
from langchain_gigachat import GigaChatEmbeddings
from langchain_chroma import Chroma

In [2]:
contacts_pd = pd.read_csv('../DataSets/contacts.csv')
schemes_pd = pd.read_csv('../DataSets/questions.csv')

In [3]:
schemes_pd

,type_question,question,format_answer,full_answer
0,"Благоустройство, ЖКХ и уборка дорог",Улица Вавилова дом 22. В 3 парадной на 2 и 5 э...,шаблонный,"Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
1,"Работа мини-аппа (найти, отписаться, указать а...",как добавить приложение в сообщество? я подпис...,шаблонный,"Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
2,Образование. Детские сады и Школы,Добрый вечер. Как быстро добавляются новые сад...,"Не знаем, ищем данные в мини-аппе","Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
3,Раздельный сбор,Кибальчича 10 - убрали раздельный сбор и тепер...,"Не знаем, ищем данные в мини-аппе","Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
4,Прочее,// пользователи присылают репосты/ эмоджи/ гол...,шаблонный,1. На репост:\n\nДобрый день/ Добрый вечер/ Зд...
5,"Благоустройство, ЖКХ и уборка дорог",Когда заменят лампочки на 2 и 5 этаже в подъез...,шаблонный,"Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
6,"Работа мини-аппа (найти, отписаться, указать а...","Приложение ужасное, пользоваться невозможно, с...","Не знаем, ищем данные именно в мини-аппе","Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
7,Образование. Детские сады и Школы,Добрый вечер. Как быстро добавляются новые сад...,"Не знаем, ищем данные в мини-аппе","Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
8,Раздельный сбор,По поводу раздельного сбора. У вас указаны кон...,"Не знаем, ищем данные в мини-аппе","Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."
9,Прочее,// что-то гневное //,шаблонный,"Добрый день/ Добрый вечер/ Здравствуйте, [имя]..."


In [4]:
contacts_pd

,id,region,category,name,phones,add_phone
0,1,Адмиралтейский,Аварийно-диспетчерские службы,Дежурный помощник главы администрации,+7(812)3160050,NaN
1,2,Адмиралтейский,Аварийно-диспетчерские службы,Повреждения водопроводной уличной сети (холодн...,+7(812)3050909,NaN
2,3,Адмиралтейский,Аварийно-диспетчерские службы,Повреждения водопроводной уличной сети (горяча...,+7(812)3151313,NaN
3,4,Адмиралтейский,Аварийно-диспетчерские службы,Горячая линия ГУП «Водоканал Санкт-Петербурга»,+7(812)3050909,NaN
4,5,Адмиралтейский,Аварийно-диспетчерские службы,Горячая линия ГРО «ПетербургГаз»,+7(812)6100404,NaN
...,...,...,...,...,...,...
466,468,Центральный,Образование,Горячая линия Комитета по образованию,+7(812)5762019,NaN
467,469,Центральный,Социальная защита населения,Информационно-справочная телефонная служба Ком...,+7(812)3344144,NaN
468,470,Центральный,Социальная защита населения,Отдел социальной защиты населения Центрального...,+7(812)4174884,NaN
469,471,Центральный,Социальная защита населения,Региональная линия Социального фонда России по...,+7(812)6778717,NaN


In [1]:
from collections import defaultdict
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")

/tmp/ipykernel_60234/50178497.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")
/home/dmitr/programs/ILiveHereBot/botenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
documents_contacts = defaultdict(list)

In [91]:
for idx, row in contacts_pd.iterrows():
    documents_contacts[row['region']].append(
        Document(
            page_content=f"{row['category']} {row['name']}",
            metadata={'phone': row['phones']}
        )
    )

In [92]:
documents_contacts

defaultdict(list,
            {'Адмиралтейский': [Document(metadata={'phone': '+7(812)3160050'}, page_content='Аварийно-диспетчерские службы Дежурный помощник главы администрации'),
              Document(metadata={'phone': '+7(812)3050909'}, page_content='Аварийно-диспетчерские службы Повреждения водопроводной уличной сети (холодная вода)'),
              Document(metadata={'phone': '+7(812)3151313'}, page_content='Аварийно-диспетчерские службы Повреждения водопроводной уличной сети (горячая вода)'),
              Document(metadata={'phone': '+7(812)3050909'}, page_content='Аварийно-диспетчерские службы Горячая линия ГУП «Водоканал Санкт-Петербурга»'),
              Document(metadata={'phone': '+7(812)6100404'}, page_content='Аварийно-диспетчерские службы Горячая линия ГРО «ПетербургГаз»'),
              Document(metadata={'phone': '+7(812)6019393'}, page_content='Аварийно-диспетчерские службы Информационный центр ГУП «ТЭК СПб»'),
              Document(metadata={'phone': '+7(812)5765

In [29]:
client = chromadb.Client()

In [ ]:
collection_contacts = client.get_or_create_collection(name="Contacts")

In [ ]:
vector_store_contacts = {
    k: Chroma(embedding_function=embeddings, collection_name=collection_contacts) for k in documents_contacts.keys()
}

In [25]:
for k in documents_contacts.keys():
    vector_store_contacts[k].add_documents(documents_contacts[k])

In [26]:
retriever = vector_store_contacts['Адмиралтейский'].as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

KeyError: 'Адмиралтейский'

In [27]:
retriever.invoke('Где оформить ОМС?')

NameError: name 'retriever' is not defined

Grad (cloud)

Gosuslugi, История,

In [32]:
client = chromadb.Client()

In [33]:
collection_themes = client.get_or_create_collection(name="collection_themes")

In [34]:
vector_store_themes = Chroma(embedding_function=embeddings)

In [35]:
vector_store_themes.add_documents([Document('Поиск контактов, основанный на Базе Контактов Санкт-Петербурга'), 
                                    Document('Образование, Детские сады и Школы'), 
                                    Document('Благоустройство, ЖКХ и уборка дорог'),
                                    Document('Афиша и Красивые места')])

['97a7a019-4877-4570-bec4-f2955d00a51e',
 'f3543af9-34f1-41d0-9e1f-e3f99910b30b',
 '9d8b12b7-38ad-4afe-ba1b-c706b8d2909a',
 '319e77d1-869f-45cd-8779-a69adbd7947c']

In [36]:
retriever_2 = vector_store_themes.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

In [37]:
retriever_2.invoke('Как позвонить в ближайший хостел')

[Document(metadata={}, page_content='Поиск контактов, основанный на Базе Контактов Санкт-Петербурга')]

In [ ]:
from collections import defaultdict
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

class typeDefiner():
    def __init__(self):
        client = chromadb.Client()
        collection_themes = client.get_or_create_collection(name="collection_themes")
        
        embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")
        self.vector_store_themes = Chroma(embedding_function=embeddings)
        self.vector_store_themes.add_documents([Document('Поиск контактов, основанный на Базе Контактов Санкт-Петербурга'), 
                                        Document('Образование, Детские сады и Школы'), 
                                        Document('Благоустройство, ЖКХ и уборка дорог'),
                                        Document('Афиша и Красивые места')])
    def define_type(self, query: str):
        self.retriever_2 = self.vector_store_themes.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 1})
        
        return retriever_2.invoke(query)[0].page_content

In [45]:
definer = typeDefiner()

In [47]:
definer.define_type('Что делать при протёкшей крыше?')

'Благоустройство, ЖКХ и уборка дорог'